# Searching for String Flood in Large Datasets on NYC Open Data
Author: Mark Bauer

# Importing Libraries

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from sodapy import Socrata

Documention for installing watermark: https://github.com/rasbt/watermark.

In [2]:
%reload_ext watermark
%watermark -t -d -v -p pandas,sodapy

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

pandas: 1.4.3
sodapy: 2.1.1



# Socrata API
I used the Socrata API to retrieve metadata for datasets hosted on NYC Open Data. Documentation can be found here: https://dev.socrata.com/. Additionally, I used sodapy, the python client for the Socrata API, to query the metadata.

### Note:  
`WARNING:root:Requests made without an app_token will be subject to strict throttling limits.`

Read more from the SODA documentation here: https://dev.socrata.com/docs/app-tokens.html

In [3]:
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# initialize Socrata object to fetch data
client = Socrata(
    domain=socrata_domain,
    app_token=None,
    timeout=1000
)

print(client)

In [22]:
# build dictionary of dataset and id
datasets = {
    '311 Service Requests from 2010 to Present':'erm2-nwe9',
    'Emergency Response Incidents':'pasr-j7fb',
    'NYCEM Emergency Notifications':'8vv7-7wx3',
    'Incidents Responded to by Fire Companies':'tm6d-hbzd',
    'Street Construction Permits (2013-2021)':'c9sj-fmsg',
    'Street Construction Permits (2022-Present)':'tqtj-sjs8',
}

datasets  

{'311 Service Requests from 2010 to Present': 'erm2-nwe9',
 'Emergency Response Incidents': 'pasr-j7fb',
 'NYCEM Emergency Notifications': '8vv7-7wx3',
 'Incidents Responded to by Fire Companies': 'tm6d-hbzd',
 'Street Construction Permits (2013-2021)': 'c9sj-fmsg',
 'Street Construction Permits (2022-Present)': 'tqtj-sjs8'}

# Query Datasets
Search for word `flood` in the dataset.

## 311 Service Requests from 2010 to Present

In [5]:
socrata_dataset_identifier = datasets['311 Service Requests from 2010 to Present']

query = """
SELECT 
    descriptor, 
    count(descriptor) as count
WHERE 
    LOWER(descriptor) LIKE '%flood%'   
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df

shape of data: (13, 2)


,descriptor,count
0,Catch Basin Clogged/Flooding (Use Comments) (SC),112454
1,Street Flooding (SJ),38187
2,Flood Light Lamp Out,6486
3,Highway Flooding (SH),3133
4,Flood Light Lamp Cycling,2591
5,Flooding on Street,673
6,Ready NY - Flooding,271
7,Flood Light Lamp Dayburning,225
8,Flood Light Lamp Missing,211
9,Flood Light Lamp Dim,185


Example:

In [6]:
socrata_dataset_identifier = datasets['311 Service Requests from 2010 to Present']

query = """
SELECT 
    *
WHERE 
    descriptor = 'Street Flooding (SJ)'   
LIMIT
    50000
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head()

shape of data: (38187, 32)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,...,latitude,longitude,location,closed_date,resolution_description,resolution_action_updated_date,intersection_street_1,intersection_street_2,facility_type,due_date
0,59921639,2024-01-04T17:41:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11414,164-40 99 STREET,99 STREET,164 AVE,...,40.650884987043256,-73.8320413316199,"{'latitude': '40.650884987043256', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59922948,2024-01-04T13:41:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11204,5901 19 AVENUE,19 AVENUE,59 ST,...,40.621198202891556,-73.98456459773816,"{'latitude': '40.621198202891556', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59921638,2024-01-04T12:23:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,EAST 103 STREET,EAST 103 STREET,AVENUE N,...,NaN,NaN,NaN,2024-01-04T12:30:00.000,The Department of Environmental Protection has...,2024-01-04T12:30:00.000,NaN,NaN,NaN,NaN
3,59924254,2024-01-04T11:47:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11375,108-36 63 ROAD,63 ROAD,108 ST,...,40.73462429006765,-73.84916828941955,"{'latitude': '40.73462429006765', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59928357,2024-01-04T10:08:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,1445 EAST 103 STREET,EAST 103 STREET,AVENUE N,...,40.63753744604451,-73.88609231636039,"{'latitude': '40.63753744604451', 'longitude':...",2024-01-04T10:20:00.000,The Department of Environmental Protection has...,2024-01-04T10:20:00.000,NaN,NaN,NaN,NaN


## Emergency Response Incidents

In [7]:
socrata_dataset_identifier = datasets['Emergency Response Incidents']

query = """
SELECT 
    incident_type, 
    count(incident_type) as count
WHERE 
    LOWER(incident_type) LIKE '%flood%'   
GROUP BY 
    incident_type
ORDER BY 
    count(incident_type) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df

shape of data: (7, 2)


,incident_type,count
0,Weather-Flooding,42
1,Other-Henry Hudson Pkwy Flooding Condition,1
2,Structural-Basement Flooding,1
3,Other-Flooding Condition,1
4,Utility-Flooding in Con Ed Substation,1
5,Weather-Coastal Flooding,1
6,Transportation-Roadway Flooding,1


Example:

In [8]:
socrata_dataset_identifier = datasets['Emergency Response Incidents']

query = """
SELECT 
    *
WHERE 
    incident_type = 'Weather-Flooding'   
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head()

shape of data: (42, 7)


,incident_type,location,borough,creation_date,latitude,longitude,closed_date
0,Weather-Flooding,61-20 Grand Central Parkway,Queens,2021-09-03T17:34:09.000,40.73831609467984,-73.84677660196152,NaN
1,Weather-Flooding,NaN,Brooklyn,2018-07-17T16:26:09.000,NaN,NaN,2018-07-17T17:41:26.000
2,Weather-Flooding,73 Place & Cooper Avenue,Queens,2012-09-08T15:45:01.000,40.70399570604266,-73.87709552817279,2012-09-08T16:45:44.000
3,Weather-Flooding,40 Sioux Street,Staten Island,2012-12-27T11:13:51.000,40.57567270482992,-74.08503664729585,2012-12-27T12:52:01.000
4,Weather-Flooding,Bell Blvd. & Northern Blvd,Queens,2019-12-14T08:13:45.000,40.760279999999995,-73.7693685,2019-12-14T11:07:08.000


## NYCEM Emergency Notifications

In [9]:
socrata_dataset_identifier = datasets['NYCEM Emergency Notifications']

query = """
SELECT 
    notification_title, 
    count(notification_title) as count
WHERE 
    LOWER(notification_title) LIKE '%flood%'   
GROUP BY 
    notification_title
ORDER BY 
    count(notification_title) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(20)

shape of data: (970, 2)


,notification_title,count
0,Flood Advisory (CW),11
1,Coastal Flood Advisory - Queens South,9
2,Flood Advisory - The Bronx,8
3,Coastal Flood Statement (S.QN),8
4,Flash Flood Watch (CW),8
5,Coastal Flood Advisory (S.QN),6
6,Coastal Flood Statement - Queens South,6
7,Flood Advisory (QN),6
8,Flash Flood Warning (CW),6
9,Coastal Flood Statement - Manhattan,5


Example:

In [10]:
socrata_dataset_identifier = datasets['NYCEM Emergency Notifications']

query = """
SELECT 
    *
WHERE 
    LOWER(notification_title) LIKE '%flood%' 
LIMIT
    2000
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(10)

shape of data: (1139, 5)


,record_id,date_and_time,notificationtype,notification_title,email_body
0,2007,2012-06-06T21:08:00.000,Weather,Flash Flood Warning Staten Island,Notification issued 6/6/12 at 9:08 PM. The Nat...
1,2162,2012-07-18T16:40:00.000,Weather,"Flash Flood Warning (QN,MN,BX,BK)",Notification issued 7/18/12 at 4:40 PM. The Na...
2,2206,2012-07-28T16:07:00.000,Weather,"Flash Flood Warning (QN,MN,BX,BK)",Notification issued 7/28/12 at 4:07 PM. The Na...
3,2217,2012-08-01T12:02:00.000,Weather,"Flash Flood Warning (QN,BK,SI)",Notification issued 8/1/12 at 12:02 PM. The Na...
4,2218,2012-08-01T12:30:00.000,Transportation,Road Closure - Flooding - LIE at Francis Lewis...,Notification issued 8/1/12 at 12:30 PM. Due to...
5,2220,2012-08-01T16:15:00.000,Transportation,Road Closure - Flooding -(QN),Notification issued 8/1/12 at 4:10 PM. Followi...
6,2263,2012-08-15T13:35:00.000,Weather,Flash Flood / Severe Thunderstorm Warnings (QN...,Notification issued 8/15/12 at 1:35 PM. The Na...
7,2266,2012-08-15T15:55:00.000,Weather,Flash Flood & Severe Thunderstorm Warnings Iss...,Notification issued 8/15/12 at 3:55 PM. The Na...
8,2267,2012-08-15T16:40:00.000,Weather,"Flash Flood Warning - BK, QN (SMS Only)",NaN
9,2351,2012-09-08T12:40:00.000,Transportation,Roadway Flooding w/ Lane Closures (QN),Notification issued 9/8/12 at 12:40 PM. Due to...


## Incidents Responded to by Fire Companies

In [11]:
socrata_dataset_identifier = datasets['Incidents Responded to by Fire Companies']

query = """
SELECT 
    incident_type_desc, 
    count(incident_type_desc) as count
WHERE 
    LOWER(incident_type_desc) LIKE '%flood%'   
GROUP BY 
    incident_type_desc
ORDER BY 
    count(incident_type_desc) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df

shape of data: (1, 2)


,incident_type_desc,count
0,812 - Flood assessment,263


Example:

In [12]:
socrata_dataset_identifier = datasets['Incidents Responded to by Fire Companies']

query = """
SELECT 
    *
WHERE 
    incident_type_desc = '812 - Flood assessment'   
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head()

shape of data: (263, 18)


,im_incident_key,fire_box,incident_type_desc,incident_date_time,arrival_date_time,units_onscene,last_unit_cleared_date_time,highest_level_desc,total_incident_duration,action_taken1_desc,property_use_desc,street_highway,zip_code,borough_desc,action_taken2_desc,action_taken3_desc,floor,co_detector_present_desc
0,55688773,2375,812 - Flood assessment,2013-01-07T07:14:45.000,2013-01-07T07:22:35.000,1,2013-01-07T07:33:12.000,"1 - More than initial alarm, less than Signal 7-5",1107,21 - Search,429 - Multifamily dwelling,E 162 ST,10459,2 - Bronx,NaN,NaN,NaN,NaN
1,55701874,2891,812 - Flood assessment,2013-01-11T18:53:20.000,2013-01-11T19:02:21.000,1,2013-01-11T19:08:37.000,"1 - More than initial alarm, less than Signal 7-5",917,21 - Search,"500 - Mercantile, business, other",WESTCHESTER AVE,10472,2 - Bronx,55 - Establish safe area,86 - Investigate,1,NaN
2,55738238,1629,812 - Flood assessment,2013-01-25T16:39:31.000,2013-01-25T16:51:39.000,1,2013-01-25T16:57:05.000,"1 - More than initial alarm, less than Signal 7-5",1054,82 - Notify other agencies.,"962 - Residential street, road or residential ...",CIRCLE RD,10304,3 - Staten Island,NaN,NaN,NaN,NaN
3,55768666,1034,812 - Flood assessment,2013-02-04T17:59:44.000,2013-02-04T18:06:59.000,1,2013-02-04T18:14:42.000,"1 - More than initial alarm, less than Signal 7-5",898,86 - Investigate,429 - Multifamily dwelling,PARK AVE,10021,1 - Manhattan,81 - Incident command,NaN,NaN,NaN
4,55786874,9114,812 - Flood assessment,2013-02-11T09:19:39.000,NaN,NaN,2013-02-11T09:22:34.000,"1 - More than initial alarm, less than Signal 7-5",175,86 - Investigate,419 - 1 or 2 family dwelling,73 TER,11367,5 - Queens,NaN,NaN,NaN,NaN


## Street Construction Permits (2013-2021)

In [13]:
socrata_dataset_identifier = datasets['Street Construction Permits (2013-2021)']

query = """
SELECT 
    permitpurposecomments, 
    count(permitpurposecomments) as count
WHERE 
    LOWER(permitpurposecomments) LIKE '%flood%'   
GROUP BY 
    permitpurposecomments
ORDER BY 
    count(permitpurposecomments) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(20)

shape of data: (365, 2)


,permitpurposecomments,count
0,"Installation of Sanitary and Storm Sewers, Flo...",562
1,"Ext. masonry, Prpts, Roof, Flood Elim, TUC rem...",318
2,FURNISH AND INSTALL FLOOD PROTECTION DEVICES A...,279
3,"Ext Masory, Flood Elim, Roofs,Windows",240
4,"Prpts, Etx. Masonry, Flood Elim, Security Light",156
5,"PS148k - Flood Elim, Ext Masonry, Parapets",154
6,MTA Contract: TUN-MIT-01: Design-Build for Flo...,143
7,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,134
8,148th Street Yard Flood Mitigation,132
9,Furnishing and installing Flood Barriers,129


In [14]:
socrata_dataset_identifier = datasets['Street Construction Permits (2013-2021)']

query = """
SELECT 
    permitpurposecomments, 
    count(permitpurposecomments) as count
WHERE 
    LOWER(permitpurposecomments) LIKE '%flood barrier%'   
GROUP BY 
    permitpurposecomments
ORDER BY 
    count(permitpurposecomments) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(20)

shape of data: (38, 2)


,permitpurposecomments,count
0,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,134
1,Furnishing and installing Flood Barriers,129
2,installing flood barriers,88
3,Intalling flood barriers,73
4,Furnishing and Installing Flood Barriers,70
5,"MTA contract- install flood barrier, marine do...",68
6,Flood Barrier Work at Astoria Houses Buildings...,40
7,Furnishing and installing flood barriers,16
8,Installation of flood barriers around Con Ed v...,14
9,Installing Flood Barriers,12


Example:

In [15]:
results_df.iloc[0].values[0]

'DEPLOY / INSTALL FOLDABLE FLOOD BARRIER'

In [16]:
socrata_dataset_identifier = datasets['Street Construction Permits (2013-2021)']

query = """
SELECT 
    *
WHERE 
    permitpurposecomments = 'DEPLOY / INSTALL FOLDABLE FLOOD BARRIER'   
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head()

shape of data: (134, 27)


,permitnumber,applicationtrackingid,sequencenumber,applicationtypeshortdesc,permitstatusid,permitstatusshortdesc,permitseriesid,permitseriesshortdesc,permittypeid,permittypedesc,...,fromstreetname,permitteename,permitpurposecomments,sidewalkshortdesc,createdon,modifiedon,oftcode,equipmenttypedesc,tostreetname,previouspermitnumber
0,M022021215B09,2021080200764016,0001,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0201,PLACE MATERIAL ON STREET,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
1,M022021215B10,2021080200764016,0002,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0211,OCCUPANCY OF ROADWAY AS STIPULATED,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
2,M022021215B11,2021080200764016,0003,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0215,OCCUPANCY OF SIDEWALK AS STIPULATED,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
3,M022021215B12,2021080200764016,0004,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0204,PLACE EQUIPMENT OTHER THAN CRANE OR SHOV,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,Backhoe,NaN,NaN
4,M022021215B13,2021080200764016,0005,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0204,PLACE EQUIPMENT OTHER THAN CRANE OR SHOV,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,Boom Truck,NaN,NaN


## Street Construction Permits (2022-Present)

In [17]:
socrata_dataset_identifier = datasets['Street Construction Permits (2022-Present)']

query = """
SELECT 
    permitpurposecomments, 
    count(permitpurposecomments) as count
WHERE 
    LOWER(permitpurposecomments) LIKE '%flood%'   
GROUP BY 
    permitpurposecomments
ORDER BY 
    count(permitpurposecomments) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(20)

shape of data: (105, 2)


,permitpurposecomments,count
0,"Installation of Sanitary and Storm Sewers, Flo...",996
1,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,570
2,MTA Contract C-33941 - Flood Mitigation and Ro...,220
3,"Windows, Ext Masonry, Roofs, Prpts. Flood elim.",192
4,"Flood Elim, Ext. Masonry, Parapets.",145
5,"Exterior Masonry, Roof, Parapet, Flood Elimina...",141
6,"Windows, Ext masonry, Roofs, Pprts, Flood elim.",128
7,"PLA Atl- Windows, Flood Elim, Ext Masonry",121
8,"Ext Masory, Flood Elim, Roofs,Windows",120
9,FURNISH AND INSTALL FLOOD PROTECTION DEVICES A...,119


In [18]:
socrata_dataset_identifier = datasets['Street Construction Permits (2022-Present)']

query = """
SELECT 
    permitpurposecomments, 
    count(permitpurposecomments) as count
WHERE 
    LOWER(permitpurposecomments) LIKE '%flood barrier%'   
GROUP BY 
    permitpurposecomments
ORDER BY 
    count(permitpurposecomments) DESC
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head(20)

shape of data: (7, 2)


,permitpurposecomments,count
0,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,570
1,"MTA contract- install flood barrier, marine do...",24
2,Flood Barrier Work at Astoria Houses Buildings...,20
3,Test deployment of flood barriers on West Street,6
4,Construct flood barriers on north side of Libe...,3
5,Test deployment of flood barriers,3
6,Installation of flood barriers around Con Ed v...,1


Example:

In [19]:
results_df.iloc[0].values[0]

'DEPLOY / INSTALL FOLDABLE FLOOD BARRIER'

In [20]:
socrata_dataset_identifier = datasets['Street Construction Permits (2013-2021)']

query = """
SELECT 
    *
WHERE 
    permitpurposecomments = 'DEPLOY / INSTALL FOLDABLE FLOOD BARRIER'   
"""

results = client.get(socrata_dataset_identifier, query=query)
results_df = pd.DataFrame(results)

print('shape of data: {}'.format(results_df.shape))
results_df.head()

shape of data: (134, 27)


,permitnumber,applicationtrackingid,sequencenumber,applicationtypeshortdesc,permitstatusid,permitstatusshortdesc,permitseriesid,permitseriesshortdesc,permittypeid,permittypedesc,...,fromstreetname,permitteename,permitpurposecomments,sidewalkshortdesc,createdon,modifiedon,oftcode,equipmenttypedesc,tostreetname,previouspermitnumber
0,M022021215B09,2021080200764016,0001,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0201,PLACE MATERIAL ON STREET,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
1,M022021215B10,2021080200764016,0002,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0211,OCCUPANCY OF ROADWAY AS STIPULATED,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
2,M022021215B11,2021080200764016,0003,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0215,OCCUPANCY OF SIDEWALK AS STIPULATED,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,NaN,NaN,NaN
3,M022021215B12,2021080200764016,0004,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0204,PLACE EQUIPMENT OTHER THAN CRANE OR SHOV,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,Backhoe,NaN,NaN
4,M022021215B13,2021080200764016,0005,New,030,EXPIRED,02,BUILDING OPERATION PERMIT,0204,PLACE EQUIPMENT OTHER THAN CRANE OR SHOV,...,VESTRY STREET,TULLY CONSTRUCTION CO. INC.,DEPLOY / INSTALL FOLDABLE FLOOD BARRIER,CONCRETE,2021-08-03T09:57:58.000,2021-10-01T00:05:10.000,123190132710000000,Boom Truck,NaN,NaN


In [21]:
client.close()